<a href="https://colab.research.google.com/github/aworklab/everydayPython/blob/main/ex_20220724.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 목표
* customer analysis
* iterator

In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime, date, time, timezone, timedelta
import os 

from functools import reduce
from collections import ChainMap, defaultdict,Counter

# customer analysis

In [2]:
os.listdir()

['.config',
 'listings.csv',
 'rfm_test.csv',
 '.Rhistory',
 'KPMG_VI_New_raw_data_update_final.xlsx',
 'test.csv',
 'sample_submission.csv',
 'ecommerce-data.csv',
 'online_retail_II.xlsx',
 'retail_data_transactions.csv',
 'retail_data_response.csv',
 'items.csv',
 'sales_train.csv',
 'shops.csv',
 'item_categories.csv',
 'sample_data']

In [10]:
df = pd.read_csv('ecommerce-data.csv', encoding = 'unicode-escape')
df.columns = [a.lower() for a in df.columns]
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [14]:
df = df[(~df.invoiceno.astype(str).str.contains('C'))&(df.country == 'United Kingdom')]
df.invoicedate = pd.to_datetime(df.invoicedate)
df['date'] = df.invoicedate.to_numpy().astype('datetime64[D]')
df['month'] = df.invoicedate.to_numpy().astype('datetime64[M]')
df['rev'] = df.quantity * df.unitprice
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,date,month,rev
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,2010-12-01,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,2010-12-01,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-12-01,20.34


In [16]:
df = df[['customerid', 'date', 'month', 'rev']].rename(columns = {'customerid':'id'})
df = df.groupby(['id','date','month'])['rev'].sum().reset_index()
df.head()

,id,date,month,rev
0,12346.0,2011-01-18,2011-01-01,77183.60
1,12747.0,2010-12-05,2010-12-01,358.56
2,12747.0,2010-12-13,2010-12-01,347.71
3,12747.0,2011-01-20,2011-01-01,303.04
4,12747.0,2011-03-01,2011-03-01,310.78


In [18]:
info_m = df.groupby(['id','month'])['rev'].sum().reset_index().rename(columns = {'rev':'money'})
info_m = pd.pivot_table(info_m, index = 'id', columns = 'month', values = 'money').reset_index()
info_m = pd.melt(info_m, id_vars = 'id', var_name = 'month', value_name = 'money')
info_m = info_m.groupby('id').head(3)
info_m = info_m.groupby('id')['money'].sum().reset_index()
info_m.head()

,id,money
0,12346.0,77183.60
1,12747.0,1009.31
2,12748.0,5036.54
3,12749.0,0.00
4,12820.0,170.46


In [19]:
info_f = df.groupby(['id','month'])['rev'].count().reset_index().rename(columns = {'rev':'freq'})
info_f = pd.pivot_table(info_f, index = 'id', columns = 'month', values = 'freq').reset_index()
info_f = pd.melt(info_f, id_vars = 'id', var_name = 'month', value_name = 'freq')
info_f = info_f.groupby('id').head(3)
info_f = info_f.groupby('id')['freq'].sum().reset_index()
info_f.head()

,id,freq
0,12346.0,1.0
1,12747.0,3.0
2,12748.0,22.0
3,12749.0,0.0
4,12820.0,1.0


In [21]:
date_max = datetime(2011,12,31)
date_max

datetime.datetime(2011, 12, 31, 0, 0)

In [22]:
info_r = df.groupby(['id']).agg({'date': lambda x : (date_max - x.max()).days}).reset_index().rename(columns = {'date':'recency'})
info_r.head()

,id,recency
0,12346.0,347
1,12747.0,24
2,12748.0,22
3,12749.0,25
4,12820.0,25


In [23]:
info = pd.merge(pd.merge(info_r, info_f, 'left', 'id'), info_m, 'left', 'id')
info.head()

,id,recency,freq,money
0,12346.0,347,1.0,77183.60
1,12747.0,24,3.0,1009.31
2,12748.0,22,22.0,5036.54
3,12749.0,25,0.0,0.00
4,12820.0,25,1.0,170.46


In [25]:
info = info.assign(r = pd.qcut(info.recency, 5, list('54321')),
            f = pd.qcut(info.freq.rank(method = 'first'), 5, list('12345')),
            m = pd.qcut(info.money.rank(method = 'first'), 5, list('12345'))
            )
info.head()

,id,recency,freq,money,r,f,m
0,12346.0,347,1.0,77183.60,1,4,5
1,12747.0,24,3.0,1009.31,5,5,5
2,12748.0,22,22.0,5036.54,5,5,5
3,12749.0,25,0.0,0.00,5,1,1
4,12820.0,25,1.0,170.46,5,4,4


In [26]:
info.assign(rf = info.r.astype(str) + info.f.astype(str))

,id,recency,freq,money,r,f,m,rf
0,12346.0,347,1.0,77183.60,1,4,5,14
1,12747.0,24,3.0,1009.31,5,5,5,55
2,12748.0,22,22.0,5036.54,5,5,5,55
3,12749.0,25,0.0,0.00,5,1,1,51
4,12820.0,25,1.0,170.46,5,4,4,54
...,...,...,...,...,...,...,...,...
3916,18280.0,299,0.0,0.00,1,4,4,14
3917,18281.0,202,0.0,0.00,1,4,4,14
3918,18282.0,29,0.0,0.00,5,4,4,54
3919,18283.0,25,3.0,317.90,5,5,4,55


# iterator

In [27]:
mat = [list('ABC'), list('DEF'), list('GHI')]
mat

[['A', 'B', 'C'], ['D', 'E', 'F'], ['G', 'H', 'I']]

In [28]:
for r, rows in enumerate(mat) :
  for c, letters in enumerate(mat) :
    print(f'({r},{c})-{letters}')

(0,0)-['A', 'B', 'C']
(0,1)-['D', 'E', 'F']
(0,2)-['G', 'H', 'I']
(1,0)-['A', 'B', 'C']
(1,1)-['D', 'E', 'F']
(1,2)-['G', 'H', 'I']
(2,0)-['A', 'B', 'C']
(2,1)-['D', 'E', 'F']
(2,2)-['G', 'H', 'I']


In [29]:
users = [{'mail': 'gregorythomas@gmail.com', 'name': 'Brett Holland', 'sex': 'M', 'age': 73},
 {'mail': 'hintoncynthia@hotmail.com', 'name': 'Madison Martinez', 'sex': 'F', 'age': 29},
 {'mail': 'wwagner@gmail.com', 'name': 'Michael Jenkins', 'sex': 'M', 'age': 51},
 {'mail': 'daniel79@gmail.com', 'name': 'Karen Rodriguez', 'sex': 'F', 'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]
users

[{'age': 73,
  'mail': 'gregorythomas@gmail.com',
  'name': 'Brett Holland',
  'sex': 'M'},
 {'age': 29,
  'mail': 'hintoncynthia@hotmail.com',
  'name': 'Madison Martinez',
  'sex': 'F'},
 {'age': 51,
  'mail': 'wwagner@gmail.com',
  'name': 'Michael Jenkins',
  'sex': 'M'},
 {'age': 32,
  'mail': 'daniel79@gmail.com',
  'name': 'Karen Rodriguez',
  'sex': 'F'},
 {'age': 42, 'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F'}]

In [32]:
for key in list(users[0].keys()) :
  if key == list(users[0].keys())[0] :
    res = eval(f'reduce(lambda acc, cur : acc + [cur["{key}"]], users, [])')
  else :
    res = np.c_[res, eval(f'reduce(lambda acc, cur : acc + [cur["{key}"]], users, [])')]

res

array([['gregorythomas@gmail.com', 'Brett Holland', 'M', '73'],
       ['hintoncynthia@hotmail.com', 'Madison Martinez', 'F', '29'],
       ['wwagner@gmail.com', 'Michael Jenkins', 'M', '51'],
       ['daniel79@gmail.com', 'Karen Rodriguez', 'F', '32'],
       ['ujackson@gmail.com', 'Amber Rhodes', 'F', '42']], dtype='<U25')

In [33]:
portfolio = [
    ('GOOG', 100, 490.1),
    ('IBM', 50, 91.1),
    ('CAT', 150, 83.44),
    ('IBM', 100, 45.23),
    ('GOOG', 75, 572.45),
    ('AA', 50, 23.15)
]
portfolio

[('GOOG', 100, 490.1),
 ('IBM', 50, 91.1),
 ('CAT', 150, 83.44),
 ('IBM', 100, 45.23),
 ('GOOG', 75, 572.45),
 ('AA', 50, 23.15)]

In [34]:
a = Counter()
for com, share, point in portfolio :
  a[com] += share
a

Counter({'AA': 50, 'CAT': 150, 'GOOG': 175, 'IBM': 150})

In [37]:
b = defaultdict(list)
for com, share, point in portfolio :
  b[com].append((share, point))
b

defaultdict(list,
            {'AA': [(50, 23.15)],
             'CAT': [(150, 83.44)],
             'GOOG': [(100, 490.1), (75, 572.45)],
             'IBM': [(50, 91.1), (100, 45.23)]})